In [1]:
import pandas as pd
import numpy as np
import pandasql as psql

print("Setup Done")

Setup Done


In [2]:
# Load the csv file
df = pd.read_csv('Skill_Test_Raw_File v2.csv')
df.shape

(2349, 9)

In [3]:
df.columns

Index(['Date', 'Create_Date__HubSpot_Contacts',
       'First_Deal_Created_Date__HubSpot_Contacts',
       'UW_utm_campaign__HubSpot_Contacts', 'IP_Country__HubSpot_Contacts',
       'IP_City__HubSpot_Contacts',
       'IP_State_CodeRegion_Code__HubSpot_Contacts',
       'IP_StateRegion__HubSpot_Contacts', 'IP_Timezone__HubSpot_Contacts'],
      dtype='object')

In [4]:
df.columns = df.columns.str.replace('__HubSpot_Contacts', '', regex=False)
df['IP_StateRegion'] = df['IP_StateRegion'].fillna('blank')

df.columns

Index(['Date', 'Create_Date', 'First_Deal_Created_Date', 'UW_utm_campaign',
       'IP_Country', 'IP_City', 'IP_State_CodeRegion_Code', 'IP_StateRegion',
       'IP_Timezone'],
      dtype='object')

In [5]:
# 1. Objective: Show the number of leads and deals per state region.
#   a. A lead is identified if there is a create_date.
#   b. A deal is identified if there is a first_deal_created_date.
# 2. Date Range:
#   a. Only include leads where the create_date is between 2024-07-01 and 2024-09-30 (inclusive).
#   b. Only include deals where the first_deal_created_date is between 2024-07-01 and 2024-09-30 (inclusive).
# 3. Campaign Filter: 
#   a. Only include leads and deals where the campaign name contains'2pmax'.


In [6]:
query = """
    with leads as (
        select
            IP_StateRegion as state_region,
            count(Create_Date) as leads_count
        from df
        where UW_utm_campaign like '%2pmax%'
            and Create_Date between '2024-07-01' and '2024-09-30'
        group by
            IP_State_CodeRegion_Code
    ),
    deals as (
        select
            IP_StateRegion as state_region,
            count(First_Deal_Created_Date) as deals_count
        from df
        where UW_utm_campaign like '%2pmax%'
            and First_Deal_Created_Date between '2024-07-01' and '2024-09-30'
        group by
            IP_State_CodeRegion_Code    
    )
    select 
        coalesce(leads.state_region, deals.state_region) as state_region,
        coalesce(leads.leads_count, 0) as leads,
        coalesce(deals.deals_count, 0) as deals
    from 
        leads full outer join deals
            on leads.state_region = deals.state_region
    union all
    select
        'TOTAL' as state_region,
        sum(coalesce(leads.leads_count, 0)) as leads,
        sum(coalesce(deals.deals_count, 0)) as deals
    from 
        leads full outer join deals
            on leads.state_region = deals.state_region
"""

In [7]:
result = psql.sqldf(query, locals())
print(result)

            state_region  leads  deals
0                  blank      3      1
1             california     12      7
2               colorado      1      1
3   district of columbia      1      1
4               delaware      1      1
5                florida      4      3
6                   iowa      1      0
7               illinois      1      1
8               kentucky      1      1
9                  maine      1      1
10              michigan      1      1
11             minnesota      2      1
12              missouri      2      1
13        north carolina      1      1
14            new jersey      1      0
15                nevada      1      1
16              new york     10      6
17                  ohio      1      1
18                oregon     38     27
19          pennsylvania      2      3
20          rhode island      2      1
21        south carolina      2      2
22             tennessee      2      1
23                 texas      3      2
24              virginia 

In [8]:
alternate_query = """
    with leads_and_deals as (
        select
            IP_StateRegion as state_region,            
            case
                when 
                    Create_Date is not null 
                    and Create_Date between '2024-07-01' and '2024-09-30' 
                    and UW_utm_campaign like '%2pmax%'
                then 1
                else 0
            end as is_lead,         
            case
                when 
                    First_Deal_Created_Date is not null 
                    and First_Deal_Created_Date between '2024-07-01' and '2024-09-30' 
                    and UW_utm_campaign like '%2pmax%'
                then 1
                else 0
            end as is_deal
        from df  
    )
        select
            state_region,
            sum(is_lead) as leads,
            sum(is_deal) as deals
        from leads_and_deals
        where is_lead > 0 or is_deal > 0
        group by state_region
        union all
        select
            'TOTAL' as state_region,
            sum(is_lead) as leads,
            sum(is_deal) as deals
        from leads_and_deals
"""

In [9]:
result_from_alternate_query = psql.sqldf(alternate_query, locals())
print(result_from_alternate_query)

            state_region  leads_count  deals_count
0                  blank            3            1
1             california           12            7
2               colorado            1            1
3               delaware            1            1
4   district of columbia            1            1
5                florida            4            3
6               illinois            1            1
7                   iowa            1            0
8               kentucky            1            1
9                  maine            1            1
10              michigan            1            1
11             minnesota            2            1
12              missouri            2            1
13                nevada            1            1
14            new jersey            1            0
15              new york           10            6
16        north carolina            1            1
17                  ohio            1            1
18                oregon       